In [1]:
from model import EASE
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:

df = pd.read_csv('/opt/ml/input/code/data/train/train_ratings.csv')
df["time"] = 1
df["rating"] = df["time"]
df = df.drop("time", axis = 1)

In [4]:
# user_group_dfs = list(df.groupby('user')['item'])
# first_row = True
# user_neg_dfs = pd.DataFrame()
# items = set(df["item"])
# for u, u_items in tqdm(user_group_dfs):
#     u_items = set(u_items)
#     num_negative = len(u_items)
#     i_user_neg_item = np.random.choice(list(items - u_items), num_negative, replace=False)
    
#     i_user_neg_df = pd.DataFrame({'user': [u]*num_negative, 'item': i_user_neg_item, 'rating': [0]*num_negative})
#     if first_row == True:
#         user_neg_dfs = i_user_neg_df
#         first_row = False
#     else:
#         user_neg_dfs = pd.concat([user_neg_dfs, i_user_neg_df], axis = 0, sort=False)
        
# joined_rating_df = pd.concat([df, user_neg_dfs], axis = 0, sort=False)

100%|██████████| 31360/31360 [20:42<00:00, 25.24it/s]


In [3]:
ease = EASE()
ease.fit(df)

In [4]:
answer = ease.predict(df, df["user"], df["item"],10)
answer
# answer = answer.drop("score", axis = 1)
# answer.to_csv("./submission2.csv", index = False)

100%|██████████| 31360/31360 [01:13<00:00, 429.08it/s]


,user,item,score
0,11,4370,0.737571
1,11,33004,0.611744
2,11,30707,0.594000
3,11,2997,0.590051
4,11,47,0.557925
...,...,...,...
313595,138493,4975,0.515432
313596,138493,4022,0.507874
313597,138493,919,0.502296
313598,138493,377,0.492138


In [5]:
answer_df = pd.DataFrame(ease.pred)
answer_df.columns = ease.item_enc.inverse_transform(answer_df.columns)
answer_df.index = ease.user_enc.inverse_transform(answer_df.index)

In [ ]:
answer_df.to_csv("./ease.csv", index = answer_df.index, columns=answer_df.columns)

In [11]:
ans_mini = answer_df.min().min()
ans_maxi = answer_df.max().max()-ans_mini
norm_answer_df = answer_df.apply(lambda x:(x-ans_mini)/ans_maxi)

In [47]:
def change(x):
    return 1 if x>0.5 else 0
norm_answer_df.apply(lambda x: change)


1         <function change at 0x7fe6900e0040>
2         <function change at 0x7fe6900e0040>
3         <function change at 0x7fe6900e0040>
4         <function change at 0x7fe6900e0040>
5         <function change at 0x7fe6900e0040>
                         ...                 
118700    <function change at 0x7fe6900e0040>
118900    <function change at 0x7fe6900e0040>
118997    <function change at 0x7fe6900e0040>
119141    <function change at 0x7fe6900e0040>
119145    <function change at 0x7fe6900e0040>
Length: 6807, dtype: object

In [ ]:
import lightgbm as lgb

In [ ]:
lgb.data